<a href="https://colab.research.google.com/github/weasel-codes/anomaly-detection-iot/blob/main/anomal_detection_iot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Performing basic file split using wireshark

In [1]:
!apt-get install tshark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tshark is already the newest version (2.6.10-1~ubuntu18.04.0).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
base_url = '/content/drive/MyDrive/ML/anomaly/'

# Import Dataset Description

In [4]:
file = pd.read_excel(base_url + 'dataset_description.xlsx')
desc_arr = file.to_numpy()
desc_arr.shape

(59, 5)

In [5]:
desc_arr[54:]

array([[nan, nan, 'Mirai Botnet', 'Telnet Bruteforce',
        'tcp.dstport==23 and ip.src==192.168.0.13'],
       [41.0, 'mirai-hostbruteforce-4-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.24 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [nan, nan, 'Mirai Botnet', 'Telnet Bruteforce',
        'tcp.dstport==23 and ip.src==192.168.0.24'],
       [42.0, 'mirai-hostbruteforce-5-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.13 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [nan, nan, 'Mirai Botnet', 'Telnet Bruteforce',
        'tcp.dstport==23 and ip.src==192.168.0.13']], dtype=object)

In [6]:
tot_file = desc_arr.shape[0]
for i in range(tot_file) :
    desc_arr[i][0] = i+1
    if(isinstance(desc_arr[i][1], float)) :
      desc_arr[i][1] = desc_arr[i-1][1]

desc_arr[54:]

array([[55, 'mirai-hostbruteforce-3-dec.pcap', 'Mirai Botnet',
        'Telnet Bruteforce', 'tcp.dstport==23 and ip.src==192.168.0.13'],
       [56, 'mirai-hostbruteforce-4-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.24 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [57, 'mirai-hostbruteforce-4-dec.pcap', 'Mirai Botnet',
        'Telnet Bruteforce', 'tcp.dstport==23 and ip.src==192.168.0.24'],
       [58, 'mirai-hostbruteforce-5-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.13 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [59, 'mirai-hostbruteforce-5-dec.pcap', 'Mirai Botnet',
        'Telnet Bruteforce', 'tcp.dstport==23 and ip.src==192.168.0.13']],
      dtype=object)

In [7]:
cmd = 'tshark -r {} -T fields -e frame.number -e frame.time_relative -e wlan.sa -e wlan.da -e wlan.ta -e wlan.ra -e frame.time_delta_displayed -e frame.len -E header=y -E separator=, -E quote=d -E occurrence=f > {}'
final_cmd = cmd.format('/content/drive/MyDrive/ML/anomaly/Packets/benign-dec.pcap', '/content/drive/MyDrive/ML/anomaly/benign-dec.csv')
print(final_cmd)
os.system(final_cmd)

tshark -r /content/drive/MyDrive/ML/anomaly/Packets/benign-dec.pcap -T fields -e frame.number -e frame.time_relative -e wlan.sa -e wlan.da -e wlan.ta -e wlan.ra -e frame.time_delta_displayed -e frame.len -E header=y -E separator=, -E quote=d -E occurrence=f > /content/drive/MyDrive/ML/anomaly/benign-dec.csv


512

In [10]:
cpac_file_loc = 'packet/'
attack_key = 'attack/'
all_key = 'csv/'

for i in range(1, tot_file) :
  file_name = desc_arr[i][1]
  file_rule = desc_arr[i][4]
  file_rule = file_rule.replace('\n', ' ') # to remove newline from filters

  from_loc = base_url + cpac_file_loc + file_name
  to_loc_attack = base_url + attack_key + file_name.split('.')[0]+".csv"
  to_loc_all = base_url + all_key + file_name.split('.')[0]+".csv"

  packet_to_all_command = 'tshark -r {} -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f > {}'
  packet_to_all_command = packet_to_all_command.format(from_loc, to_loc_all)

  packet_to_attack_command = 'tshark -r {} -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f {} > {}'
  packet_to_attack_command = packet_to_attack_command.format(from_loc, file_rule, to_loc_attack)

  # print(file_name, " : All : " , packet_to_all_command)
  # os.system(packet_to_all_command)
  print(file_name, " : Attack : ", packet_to_attack_command)
  os.system(packet_to_attack_command)
  break

mitm-arpspoofing-1-dec.pcap  : Attack :  tshark -r /content/drive/MyDrive/ML/anomaly/packet/mitm-arpspoofing-1-dec.pcap -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f eth.addr == f0:18:98:5e:ff:9f and  ((((ip.src == 192.168.0.16 and ip.dst == 192.168.0.13)  or (ip.src == 192.168.0.13 and ip.dst == 192.168.0.16))  and !icmp and tcp)  or (arp.src.hw_mac == f0:18:98:5e:ff:9f  and (arp.dst.hw_mac == bc:1c:81:4b:ae:ba or arp.dst.hw_mac == 48:4b:aa:2c:d8:f9))) > /content/drive/MyDrive/ML/anomaly/attack/mitm-arpspoofing-1-dec.csv
